In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## With Tiling

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt


In [ ]:
!python /content/yolov5/export.py --weights yolov5s.pt --include torchscript --img 640


In [ ]:
import torch
from torchvision import transforms
from PIL import Image, ImageDraw
from glob import glob
import os

# Define the path to the pre-trained model file
model_path = "/content/yolov5s.pt"

# Load the YOLOv5 model from the specified file
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Set the model to evaluation mode
model.eval()

# Define the path to your image directory
image_dir = '/content/drive/MyDrive/Images/Dataset_internship-20230413T104907Z-001/Dataset_internship/*.jpeg'

# Loop over all images in the directory
for image_path in glob(image_dir):
    # Load the image and convert it to RGB format
    image = Image.open(image_path).convert('RGB')
    
    # Use the YOLOv5 model to detect objects in the image
    results = model(image)
    
    # Extract the bounding boxes and class labels from the results
    boxes = results.xyxy[0][:, :4].tolist()
    labels = results.xyxyn[0][:, -1].tolist()
    
    # Filter the results to only include "person" class labels
    person_boxes = [boxes[i] for i in range(len(labels)) if labels[i] == 0]
    
    # Draw the bounding boxes on the image
    draw = ImageDraw.Draw(image)
    for box in person_boxes:
        draw.rectangle(box, outline='red')
    
    # Save the image with the bounding boxes overlaid
    image.save('/content/drive/MyDrive/Images/' + os.path.basename(image_path))


## Without Tiling

In [ ]:
import torch
from torchvision import transforms
from PIL import Image, ImageDraw
from glob import glob
import os

# Define the path to the pre-trained model file
model_path = "/content/yolov5s.pt"

# Load the YOLOv5 model from the specified file
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Set the model to evaluation mode
model.eval()

# Define the path to your image directory
image_dir = '/content/drive/MyDrive/Images/Dataset_internship-20230413T104907Z-001/Dataset_internship/*.jpeg'

# Define the number of tiles in each dimension
num_tiles_x = 4
num_tiles_y = 3

# Loop over all images in the directory
for image_path in glob(image_dir):
    # Load the image and convert it to RGB format
    image = Image.open(image_path).convert('RGB')
    
    # Compute the size of each tile
    tile_width = image.width // num_tiles_x
    tile_height = image.height // num_tiles_y
    
    # Loop over each tile and apply the person detection model
    for i in range(num_tiles_x):
        for j in range(num_tiles_y):
            # Compute the coordinates of the tile
            x1 = i * tile_width
            y1 = j * tile_height
            x2 = (i + 1) * tile_width
            y2 = (j + 1) * tile_height
            
            # Crop the tile from the image
            tile = image.crop((x1, y1, x2, y2))
            
            # Use the YOLOv5 model to detect objects in the tile
            results = model(tile)
            
            # Extract the bounding boxes and class labels from the results
            boxes = results.xyxy[0][:, :4].tolist()
            labels = results.xyxyn[0][:, -1].tolist()
            
            # Filter the results to only include "person" class labels
            person_boxes = [boxes[k] for k in range(len(labels)) if labels[k] == 0]
            
            # Draw the bounding boxes on the tile
            draw = ImageDraw.Draw(tile)
            for box in person_boxes:
                draw.rectangle(box, outline='red')
            
            # Save the tile with the bounding boxes overlaid
            tile.save('/content/drive/MyDrive/Images/with_tilling/' + os.path.basename(image_path) + '_tile{}_{}.jpeg'.format(i, j))
